In [1]:
%matplotlib widget

In [2]:
import opticalglass as og
import opticalglass.schott as schott
from opticalglass.opticalmedium import OpticalMedium, glass_encode, glass_decode
from opticalglass.buchdahl import Buchdahl2
import pandas as pd

In [3]:
bk7 = schott.SchottGlass('N-BK7')

In [4]:
vars(bk7)

{'gname': 'N-BK7',
 'coefs': array([1.03961212e+00, 2.31792344e-01, 1.01046945e+00, 6.00069867e-03,
        2.00179144e-02, 1.03560653e+02])}

In [5]:
bk7.catalog

In [6]:
# --- material definitions
class ConstantIndex(OpticalMedium):
    """ Constant refractive index medium. """

    def __init__(self, nd, lbl, cat=''):
        self.label = lbl
        self.n = nd
        self._catalog_name = cat

    def __repr__(self):
        return ('Medium(' + str(self.n) + ', ' + f"'{self.label}'" +
                ', cat=' + f"'{self._catalog_name}'" + ')')

    def name(self):
        return self.label

    def catalog_name(self):
        return self._catalog_name

    def calc_rindex(self, _):
        return self.n

    def meas_rindex(self, _):
        return self.n

In [7]:
n2 = ConstantIndex(2., '2.0 refractive index', cat='user')

In [8]:
isinstance(n2, OpticalMedium)

True

In [9]:
hasattr(n2, 'rindex')

True

In [10]:
class ModelGlass(OpticalMedium):
    """ Optical medium defined by a glass code, i.e. index - V number pair """

    def __init__(self, nd: float, vd: float, mat: str, cat: str='user'):
        self.label = mat
        self._catalog_name = cat
        self.n = nd
        self.v = vd
        self.bdhl_model = Buchdahl2(self.n, self.v)

    def __str__(self):
        return 'ModelGlass ' + self.label + ': ' + glass_encode(self.n, self.v)

    def __repr__(self):
        return ('ModelGlass(nd=' + str(self.n) +
                ', vd=' + str(self.v) +
                ', mat=' + f"'{self.label}'" +
                ', cat=' + f"'{self._catalog_name}'" + ')')

    def sync_to_restore(self):
        if not hasattr(self, 'bdhl_model'):
            self.bdhl_model = Buchdahl2(self.n, self.v)

    def glass_code(self):
        return glass_encode(self.n, self.v)

    def name(self):
        if self.label == '':
            return glass_encode(self.n, self.v)
        else:
            return self.label

    def catalog_name(self):
        return self._catalog_name

    def calc_rindex(self, wv_nm):
        return self.bdhl_model.calc_rindex(wv_nm)

    def meas_rindex(self, wvl):
        return self.rindex(wvl)

    def update(self, nd, vd):
        self.n = nd
        self.v = vd
        self.bdhl_model.update_model(nd, vd)

In [11]:
my_bk7=ModelGlass(1.517, 64.2, 'my_bk7', cat='user')

In [12]:
type(my_bk7)

__main__.ModelGlass

In [13]:
isinstance(my_bk7, OpticalMedium)

True

In [14]:
hasattr(my_bk7, 'rindex')

True

In [15]:
bk7.rindex(pd.Index(['F', 'd', 'C']))

array([1.52237629, 1.51680003, 1.51432235])

In [16]:
bk7.meas_rindex(pd.Index(['F', 'd', 'C']))

F    1.52238
d     1.5168
C    1.51432
Name: N-BK7, dtype: object

In [17]:
my_bk7.meas_rindex(pd.Index(['F', 'd', 'C']))

array([1.52218406, 1.517     , 1.51458912])

In [18]:
bk7.glass_code()

'517.642'

In [19]:
my_bk7.glass_code()

'517.642'